These are objects that you'll inject into your tests and set up with the right values to simulate real parts of the system.

In [1]:
from unittest.mock import Mock
from pytest import raises

In [2]:
# You can create a Mock object and set it up with values to retrieve later
mock_talk = Mock(conference='EuroPython')
mock_talk.conference

'EuroPython'

In [3]:
# All properties of a mock are also mocks
mock_talk.speaker

<Mock name='mock.speaker' id='140497910462392'>

In [4]:
# And so are their properties - it's mocks all the way down.
mock_talk.speaker.name

<Mock name='mock.speaker.name' id='140497910200808'>

In [5]:
# Until you assign something else to them
mock_talk.speaker.name = 'Helen'
mock_talk.speaker.name

'Helen'

In [6]:
# Mocks are callable, and still return a mock
mock_talk.speaker.get_twitter_status()

<Mock name='mock.speaker.get_twitter_status()' id='140497910272128'>

In [7]:
# You can use return_value to tell it what to return
mock_talk.speaker.get_twitter_status.return_value = 'Eating cheese'
mock_talk.speaker.get_twitter_status()

'Eating cheese'

## Side effects

Side effects define what happens when you call a mock.

In [8]:
# A side effect can be an exception
mock_func = Mock()
mock_func.side_effect = KeyError

with raises(KeyError):
    mock_func()

In [9]:
# Or a function
mock_func.side_effect = lambda x: x*2
mock_func(2)

4

In [10]:
# It can be a list, so the mock will do a sequence of different things.
# In the cases where there's just a value, that value is returned
# (You can't have that as a single side effect though)
mock_func.side_effect = [1, ValueError, 2]
mock_func()

1

In [11]:
# The second call will give us an error
with raises(ValueError):
    mock_func()

In [12]:
# Then we'll get the next value
mock_func()

2

In [13]:
# Then it raises StopIteration because we ran out of side effects.
with raises(StopIteration):
    mock_func()

## What was called?


In [14]:
from unittest.mock import call
mock_func = Mock()
mock_func(1)
mock_func(2)

<Mock name='mock()' id='140497910317128'>

In [15]:
mock_func.assert_called_with(2)
mock_func.assert_has_calls([call(1), call(2)])
# both of these succeed

### Beware versions...

In [16]:
# In Python 3.5, this rightly raises an Assertion error,
# since mock_func has been called.
mock_func = Mock()
mock_func()

with raises(AssertionError):
    mock_func.assert_not_called()



In Python 3.4 assert_not_called does not exist so it returns another mock and silently succeeds

```python
>>> mock_func.assert_not_called()
<Mock name='mock.assert_not_called()' id='139758629541536'>
```

In [17]:
# assert_called comes in with Python 3.6
# Python 3.5 added checks for invalid assert* or assret* calls
# So you'll get an AttributeError
mock_func = Mock()
with raises(AttributeError):
    mock_func.assert_called()

### Matching args

In [18]:
# Use Mock.ANY if you don't care what the arg was
from unittest.mock import Mock, ANY

mock_func = Mock()
mock_func(25)
mock_func.assert_called_with(ANY)  # succeeds

In [19]:
# Use matcher objects for more control
class MultipleMatcher:
    def __init__(self, factor):
        self.factor = factor

    def __eq__(self, other):
        return other % self.factor == 0

    def __repr__(self):
        return 'Multiple of {}'.format(self.factor)
    
mock_func.assert_called_with(MultipleMatcher(5))  # succeeds

In [20]:
with raises(AssertionError):
    mock_func.assert_called_with(MultipleMatcher(4))